In [ ]:
# Add src directory to Python path for module imports
import sys
import os
sys.path.insert(0, os.path.abspath('../src'))

In [ ]:
# Imports
import os
import glob
import csv
from config import NYUV2, MAKE3D


In [ ]:
# Configuration: set dataset, split, and paths
split = 'test'
DATASET = "NYUv2"


if DATASET == "NYUv2":
    PARENT = NYUV2.experiment_folder
    data_dir = os.path.join(NYUV2.data_dir, "NYUv2", split + "_rgb")
    file_extension = '.png'
    out_csv = "summary_metrics_all_"+split+"_nyuv2.csv"
    exp_name_format = "nyuv2-*" # optional 
elif DATASET == "Make3D":
    PARENT = MAKE3D.experiment_folder
    data_dir = os.path.join(MAKE3D.data_dir, "Make3D", "Train400Img" if split == "train" else "Test134Img")
    file_extension = '.jpg'
    out_csv = "summary_metrics_all_"+split+"_make3d.csv"
    c1_out_csv = "summary_metrics_all_"+split+"_make3d_c1_error.csv"
    exp_name_format = "make3d-*"


In [ ]:
# Helper functions for parsing metrics and writing CSV

def parse_last_loss(losses_path):
    # todo: this reads whole file not super efficient
    with open(losses_path, "r", encoding="utf-8") as f:
        for line in f:
            if line.strip(): # not blank
                last = line.strip()
    return float(last)

def parse_accuracy_metrics(metrics_path):
    rms = rel = d1 = d2 = d3 = None
    with open(metrics_path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if line.lower().startswith("rms"):
                rms = float(line.split(":")[1].strip())
            elif line.lower().startswith("rel"):
                rel = float(line.split(":")[1].strip())
            elif line.lower().startswith("accuracy"):
                right = line.split(":")[1].strip()
                d1, d2, d3 = [float(x.strip()) for x in right.split(",")]
    return (rms, rel, d1, d2, d3)

def write_csv_with_average(rows, fieldnames, output_csv):
    """Write rows to CSV, adding an average row and sorting by run_id."""
    rows.sort(key=lambda r: r["run_id"])
    
    avg_row = {"run_id": "AVERAGE"}
    for col in fieldnames[1:]:
        values = [r[col] for r in rows if r[col] is not None]
        avg_row[col] = sum(values) / len(values) if values else None
    
    rows.append(avg_row)
    
    with open(output_csv, "w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(rows)
    
    print(f"Wrote {len(rows)} rows to {output_csv}")


In [ ]:
# Get list of experiment directories

# # Alternatively, look for specific file pattern
# pattern = os.path.join(PARENT, exp_name_format)
# dirs = [d for d in glob.glob(pattern) if os.path.isdir(d)] 

dirs = [d for d in os.listdir(PARENT) if os.path.isdir(os.path.join(PARENT, d))]

# Rremove repeats + sort
dirs = sorted(set(dirs))
print(len(dirs))

# Assert expected number of files
num_files = len(dirs)
expected_num_files = len([f for f in os.listdir(data_dir) if f.endswith(file_extension)])
assert num_files == expected_num_files, "Expected {expected_num_files} files, but found {num_files} files"


In [ ]:
# MAIN WORKFLOW: Compile accuracy metrics from experiment results
# Reads losses.txt and accuracy_metrics.txt from each experiment directory
# Outputs: RMS, Rel, delta1, delta2, delta3, last_loss

rows = []

for d in dirs:
    run_id = os.path.basename(d).split("_")[0].split("-")[-1]

    losses_path = os.path.join(PARENT, d, "losses.txt")
    metrics_path = os.path.join(PARENT, d, "accuracy_metrics.txt")

    if not os.path.exists(losses_path) or not os.path.exists(metrics_path):
        continue

    last_loss = parse_last_loss(losses_path)
    rms, rel, d1, d2, d3 = parse_accuracy_metrics(metrics_path)

    rows.append({
        "run_id": run_id,
        "last_loss": last_loss,
        "RMS": rms,
        "Rel": rel,
        "delta1": d1,
        "delta2": d2,
        "delta3": d3,
    })

fieldnames = ["run_id", "last_loss", "RMS", "Rel", "delta1", "delta2", "delta3"]
write_csv_with_average(rows, fieldnames, out_csv)


In [ ]:
# MAKE3D ONLY: Compute C1 error metrics
# Recomputes RMS and Rel from saved depth maps with C1 threshold (depth < 70)
# Only run this cell if DATASET == "Make3D"

import utils
import numpy as np

rows = []

for d in dirs:
    run_id = os.path.basename(d).split("_")[0].split("windowed5-")[1]

    dpt = utils.load_dpt_npy(os.path.join(PARENT, d), 'dpt')

    # compute c1 error
    _, gt_dpt = utils.load_single_sample_Make3D(img_name = "img-"+run_id+".jpg", data_dir = MAKE3D.data_dir,
                                                split=split)

    mask = (gt_dpt < 70) & np.isfinite(gt_dpt)
    gt_dpt_masked = gt_dpt[mask]
    dpt_masked = dpt[mask]
    assert np.all(gt_dpt_masked < 70)

    rms = utils.compute_RMS(dpt_masked, gt_dpt_masked)
    rel = utils.compute_AbsRel(dpt_masked, gt_dpt_masked)

    rows.append({
        "run_id": run_id,
        "RMS": rms,
        "Rel": rel,
    })

fieldnames = ["run_id", "RMS", "Rel"]
write_csv_with_average(rows, fieldnames, c1_out_csv)
